# Тема 7. Исследование зависимости плотности от параметров НСВ в Python
Задание:
Изучите материал из Notebook «Исследование зависимости плотности от параметров НСВ». Повторите примеры с исследованием поведения графика плотности от значений параметров НСВ для равномерного, экспоненциального, нормального законов распределения и проверку формул расчета математического ожидания и дисперсии для сгенерированных данных. Самостоятельно исследуйте поведение графика и расчет характеристик для другого вида непрерывного распределения (например, логнормальное, χ2-распределение, распределение Стьюдента, Фишера-Снедекора, см. Кремер §§ 4.8 – 4.9, https://docs.scipy.org/doc/scipy/reference/stats.html). Для рассматриваемого распределения оформите теоретическую часть (вставьте пояснения и расчетные формулы).
## Равномерное, экспоненциальное и нормлаьное распределения
### Импорт библиотек


In [1]:
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')

### Вспомогательные функции

In [2]:
def show_error(message, title="Ошибка"):
    plt.figure(figsize=(8, 4))
    plt.text(0.5, 0.5, message, 
            fontsize=14, ha='center', va='center', 
            transform=plt.gca().transAxes, color='red')
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.title(title, fontsize=14, color='red')
    plt.axis('off')
    plt.show()

def show_pdf(pdf, xmin, xmax, grid_size, dist_name, **kwargs):

    # Проверка корректности границ графика
    if xmin >= xmax:
        show_error('xmin ≥ xmax\nПроверьте границы графика!', 
                  'Некорректные параметры графика')
        return
    
    X = np.linspace(xmin, xmax, grid_size)
    try:
        Y = pdf(X, **kwargs)
    except ValueError as e:
        show_error(f'Ошибка в параметрах распределения:\n{str(e)}', 
                  'Некорректные параметры распределения')
        return
    
    ymax = max(Y) + 0.1
    plt.figure(figsize=(8, 4))
    plt.plot(X, Y, lw=3)
    plt.grid(ls=':')
    plt.xlabel('x', fontsize=12)
    plt.ylabel('φ(x)', fontsize=12)
    plt.xlim((xmin, xmax))
    plt.ylim((-0.05, ymax))
    title = f'Плотность распределения {dist_name}'
    plt.title(title, fontsize=14)
    plt.show()

def show_uniform_error(a, b):
    plt.figure(figsize=(8, 4))
    
    # Вычисляем границы для отображения
    x_min = min(a, b) - 1
    x_max = max(a, b) + 1
    
    # Закрашиваем некорректный интервал
    if a != b:
        plt.axvspan(min(a,b), max(a,b), alpha=0.3, color='red')
    
    # Вертикальные линии границ
    plt.axvline(x=a, color='red', linestyle='--', alpha=0.7, linewidth=2)
    plt.axvline(x=b, color='red', linestyle='--', alpha=0.7, linewidth=2)
    
    # Подписи
    plt.text((x_min + x_max)/2, 0.5, f'НЕКОРРЕКТНО:\na = {a:.1f} ≥ b = {b:.1f}', 
            fontsize=14, ha='center', va='center', 
            color='red', fontweight='bold')
    
    plt.xlim(x_min, x_max)
    plt.ylim(0, 1)
    plt.title(f'Равномерное распределение U({a:.1f},{b:.1f})', 
             fontsize=14, color='red')
    plt.xlabel('x', fontsize=12)
    plt.ylabel('φ(x)', fontsize=12)
    plt.grid(ls=':', alpha=0.5)
    plt.show()

### Равномерное распределение

Формула плотности: $$\varphi(x) = \begin{cases} 0, x \notin[a,b] \\ \frac {1} {b-a}, x \in [a,b] \end{cases}$$

Функция распределения: $$F(x) = \begin{cases} 0, x < a \\ \frac {x-a} {b-a}, x \in [a,b] \\ 1, x > b \end{cases}$$

Математическое ожидание: $$M(X)=\frac {a+b} {2}$$

Дисперсия: $$D(X)=\frac {(b-a)^2} {12}$$

In [3]:
@interact(a=(-10, 10, 0.5), b=(-10, 10, 0.5))
def plot_uniform_interact(a=0, b=1):
    # Проверка корректности параметров
    if a >= b:
        show_uniform_error(a, b)
    else:
        show_pdf(pdf=sts.uniform.pdf, xmin=a-1, xmax=b+1, grid_size=1000,
                 dist_name=f'$R({a:.1f},{b:.1f})$', loc=a, scale=b-a)
    
    print(f'Математическое ожидание M: {(a+b)/2:.3f}')
    print(f'Дисперсия D: {((b-a)**2)/12:.3f}')

interactive(children=(FloatSlider(value=0.0, description='a', max=10.0, min=-10.0, step=0.5), FloatSlider(valu…

### Экспоненциальное распределение

Формула плотности: $$\varphi(x) = \begin{cases} 0, x < 0 \\ \lambda e^{-\lambda x}, x \geq 0\end{cases}$$

Функция распределения: $$F(x) = \begin{cases} 0, x < 0 \\ 1 - e^{-\lambda x}, x \geq 0 \end{cases}$$

Математическое ожидание: $$M(X)=\frac {1} {\lambda}$$

Дисперсия: $$D(X)=\frac {1} {\lambda^2}$$

In [4]:
@interact(lmbda=(-1, 10, 0.05))
def plot_exp_interact(lmbda=1):
    # Проверка корректности параметра
    if lmbda <= 0:
        show_error(f'λ = {lmbda:.2f} ≤ 0\nУстановите λ > 0', 
                  'Некорректный параметр λ')
    else:
        show_pdf(pdf=sts.expon.pdf, xmin=0, xmax=10, grid_size=1000,
                 dist_name=f'$Exp(\\lambda={lmbda:.2f})$', scale=1/lmbda)
    
    print(f'Математическое ожидание M: {1/lmbda:.3f}')
    print(f'Дисперсия D: {1/(lmbda**2):.3f}')

interactive(children=(FloatSlider(value=1.0, description='lmbda', max=10.0, min=-1.0, step=0.05), Output()), _…

### Нормальное распределение

Формула плотности: $$\varphi(x) = \frac {1} {\sigma \sqrt{2 \pi}} e ^ {- \frac {(x-a)^2} {2\sigma^2}}$$

Математическое ожидание: $$M(X)=a$$

Дисперсия: $$D(X)= \sigma^2$$

In [5]:
@interact(a=(-5, 5, 0.5), sigma=(-1, 5, 0.01))
def plot_norm_interact(a=0, sigma=1):
    # Проверка корректности параметра
    if sigma <= 0:
        show_error(f'σ = {sigma:.2f} ≤ 0\nУстановите σ > 0', 
                  'Некорректный параметр σ')
    else:
        show_pdf(pdf=sts.norm.pdf, xmin=a-4*sigma, xmax=a+4*sigma, grid_size=1000,
                 dist_name=f'$N(a={a},\\sigma={sigma:.2f})$', loc=a, scale=sigma)
    
    print(f'Математическое ожидание M: {a:.3f}')
    print(f'Дисперсия D: {sigma**2:.3f}')

interactive(children=(FloatSlider(value=0.0, description='a', max=5.0, min=-5.0, step=0.5), FloatSlider(value=…

## Распределение Стьюдента (t-распределение)

### Теоретическая часть

Распределение Стьюдента - это непрерывное распределение, используемое в статичтике для оценки параметров генеральной совокупности при малых размерах выборки, когда стандартное отклонение неизвестно.

Функция плотности: $$\varphi(t)=\frac {Г(\frac {\nu + 1} {2})} {\sqrt{\nu \pi} Г(\frac \nu 2)} (1 + \frac {t^2} {\nu})^{-\frac {\nu + 1} {2}}$$
где:
- $\nu$ - число степеней свободы;
- $Г$ - гамма функция.

Функция распределения: $$F(t) = \frac 1 2 + t * \frac {Г(\frac {\nu + 1} {2})} {\sqrt{\nu \pi}Г(\frac {\nu} {2})} * \text{}  _2F_1 (\frac 1 2; \frac {\nu + 1} {2}; \frac 3 2; -\frac {t^2} {\nu})$$
где $_2F_1$ - гипергеометрическая функция

Свойства:
- Математическое ожидание: $M(T) = 0, \nu > 1$
- Дисперсия: $D(T)=\frac {\nu} {\nu-2}, \nu>2$

In [6]:
@interact(nu=(0, 30, 1))
def plot_student_interact(nu=5):
    # Проверка корректности параметра
    if nu < 1:
        show_error(f'ν = {nu} < 1\nУстановите ν ≥ 1', 
                  'Некорректный параметр ν')
    else:
        show_pdf(pdf=sts.t.pdf, xmin=-5, xmax=5, grid_size=1000,
                 dist_name=f'$t(\\nu={nu})$', df=nu)
    
    print(f'Математическое ожидание M: 0 (для ν > 1)')
    if nu > 2:
        print(f'Дисперсия D: {nu/(nu-2):.3f}')
    elif nu > 1:
        print(f'Дисперсия D: ∞ (для 1 < ν ≤ 2)')
    else:
        print(f'Дисперсия D: не определена (для ν = 1)')

interactive(children=(IntSlider(value=5, description='nu', max=30), Output()), _dom_classes=('widget-interact'…

## Сравнение распределения Стьюдента с нормальным распределением

In [7]:
@interact(nu=(1, 30, 1))
def compare_with_normal(nu=5):
    # Проверка корректности параметра
    if nu < 1:
        show_error(f'ν = {nu} < 1\nУстановите ν ≥ 1', 
                  'Некорректный параметр ν')
        return
    
    x = np.linspace(-5, 5, 1000)
    φ_t = sts.t.pdf(x, df=nu)
    φ_norm = sts.norm.pdf(x, loc=0, scale=1)
    
    plt.figure(figsize=(8, 4))
    plt.plot(x, φ_t, lw=3, label=f'$t(ν={nu})$')
    plt.plot(x, φ_norm, lw=3, label='$N(0,1)$', linestyle='--')
    plt.title(f'Сравнение плотностей: t-распределение и нормальное', fontsize=14)
    plt.xlabel('x', fontsize=12)
    plt.ylabel('φ(x)', fontsize=12)
    plt.grid(ls=':')
    plt.legend()
    plt.show()

interactive(children=(IntSlider(value=5, description='nu', max=30, min=1), Output()), _dom_classes=('widget-in…